Naming Explanation :
XXXYYZZZ
XXX - ID number of a person who has done the signature. 
YY - Image sample number.
ZZZ - ID number of person whose signature is in photo.

Example: 
NFI-00602023 is an image of signature of person number 023 done by person 006. This is a forged signature.
NFI-02103021 is an image of signature of person number 021 done by person 021. This is a genuine signature.   

In [2]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import ndimage
from skimage.filters import threshold_otsu
from skimage.filters import prewitt_h,prewitt_v
import tensorflow as tf
from keras import models, layers, preprocessing
import splitfolders
import os

In [3]:
import git
repo = git.Repo('.', search_parent_directories=True)

In [4]:
#Preprocessing

def rgbToGray (img_path):
    img = cv2.imread(img_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    return img_gray

def filterNoise(gray_img):
    img_filtered = ndimage.gaussian_filter(gray_img, 0.5)

    return img_filtered

def grayToBin(img_filtered):
    (thresh, binImg) = cv2.threshold(img_filtered, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # thres = threshold_otsu(resizedImg)
    # binImg = resizedImg > thres
    return binImg

def resizeImg(binImg):
    resizedImg = cv2.resize(binImg,(256, 256))
    return resizedImg

def preProcessing(image_path) :
    grayImg = rgbToGray(image_path)
    img_filtered = filterNoise(grayImg)
    binaryImg = grayToBin(img_filtered)
    resizedImg = resizeImg(binaryImg)

    return resizedImg

In [14]:
path_real = repo.working_tree_dir + "\\image\\real\\"
path_forge = repo.working_tree_dir + "\\image\\forge\\"

path_real_p = repo.working_tree_dir + "\\image_processed\\real\\"
path_forge_p = repo.working_tree_dir + "\\image_processed\\forge\\"

def preProc(path, path_p) :
    try:
        os_path = os.listdir(path)
        for img in os_path[:]:
            print("From : " + os.path.join(path, img))
            print("To : " + os.path.join(path_p, img))
            processed_img = preProcessing(os.path.join(path, img))
            # status = plt.imsave(path_p + img, processed_img, cmap="gray")
            status = cv2.imwrite(os.path.join(path_p, img), processed_img)
            print(status)
        return
    except Exception as e:
        print(f"Error : {e}")

In [15]:
preProc(path_real, path_real_p)
preProc(path_forge, path_forge_p)

From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\real\00100001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_processed\real\00100001.png
True
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\real\00101001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_processed\real\00101001.png
True
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\real\00102001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_processed\real\00102001.png
True
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\real\00103001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_processed\real\00103001.png
True
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\real\00104001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_processed\real\00104001.png
True
From : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image\real\00105001.png
To : e:\Kuliah\SMT 6\Citra\pcd-sign-recognition\image_processed\real\00105001.png
True
From : e:\Kuliah\SMT 6\Citra\pcd-s

In [16]:
#Split Folder
input_folder =  repo.working_tree_dir + "/image_processed"

splitfolders.ratio(input_folder, output="image_train_test_val", seed = 42, ratio=(.7 , 0 , .3), group_prefix=None)

In [8]:
# Features Used :
# Centroid : Center of Image.  In case of a grayscale image you can use the pixels' gray values to calculate the weighted average position.
# Ratio ? the ratio of its width to its height
# Eccentricity ? a disparity on an image between the centre of the projected object and the projected location of the centre of an object
# Solidity ? Solidity is useful to quantify the amount and size of concavities in an object boundary. Holes are also often included
# Skewness : meausre of symmetric. if the skewness is negative, the histogram is negatively skewed. The positive skewness is the opposite.
# Kurtosis : is the average (or expected value) of the standardized data raised to the fourth power.

In [9]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])
# model.summary()

In [17]:
# Build a neural network in Keras

# Function to resize image to 56x56
def resize_image(image):
    import tensorflow as tf
    return tf.image.resize(image,[56,56])

row, col, ch = 256, 256, 1

model = models.Sequential()
model.add(layers.ZeroPadding2D((1, 1), input_shape=(row, col, ch)))

# Resise data within the neural network
model.add(layers.Lambda(resize_image))  #resize images to allow for easy computation

# CNN model - Building the model suggested in paper

model.add(layers.Convolution2D(filters= 32, kernel_size =(5,5), strides= (2,2), padding='same', name='conv1')) #96
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2),strides=(2,2), name='pool1'))

model.add(layers.Convolution2D(filters= 64, kernel_size =(3,3), strides= (1,1), padding='same', name='conv2'))  #256
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2),strides=(2,2), name='pool2'))

model.add(layers.Convolution2D(filters= 128, kernel_size =(3,3), strides= (1,1), padding='same', name='conv3'))  #256
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2),strides=(2,2), name='pool3'))


model.add(layers.Flatten())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, name='dense1'))  #1024
# model.add(BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(256, name='dense2'))  #1024
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(50,name='output')) # 50? No. of classes
model.add(layers.Activation('softmax'))  #softmax since output is within 50 classes

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_1 (ZeroPaddi  (None, 258, 258, 1)      0         
 ng2D)                                                           
                                                                 
 lambda_1 (Lambda)           (None, 56, 56, 1)         0         
                                                                 
 conv1 (Conv2D)              (None, 28, 28, 32)        832       
                                                                 
 activation_6 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 pool1 (MaxPooling2D)        (None, 14, 14, 32)        0         
                                                                 
 conv2 (Conv2D)              (None, 14, 14, 64)        18496     
                                                      

In [25]:
bs=32         #Setting batch size
train_dir = repo.working_tree_dir + "\\image_train_test_val\\train"   #Setting training directory
test_dir = repo.working_tree_dir + "\\image_train_test_val\\test"   #Setting testing directory
# All images will be rescaled by 1./255.
train_datagen = preprocessing.image.ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = preprocessing.image.ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(180,180))
test_generator=test_datagen.flow_from_directory(test_dir,batch_size=bs,class_mode='categorical',target_size=(180,180))

Found 378 images belonging to 2 classes.
Found 163 images belonging to 2 classes.


In [26]:
model.fit(train_generator, batch_size=128, epochs=10, validation_data=test_generator)

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node 'sequential_1/activation_6/Relu' defined at (most recent call last):
    File "C:\Users\eugen\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\eugen\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\eugen\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\eugen\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\eugen\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\eugen\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 199, in start
      self